# Searching for Be stars in binaries with compact objects
## Catalog cross-matching
### Caden Gobat, The George Washington University

In [7]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, astropy.units as u, pickle
from astropy.io import ascii as ascii_io
from astropy.coordinates import SkyCoord, Angle
from astroquery.vizier import Vizier

### Initialize ViZier query instance

In [ ]:
v = Vizier(columns=['all', '_RAJ2000', '_DEJ2000', "_r"])
v.ROW_LIMIT = -1 # no row limit
local = True if input("If catalogs are stored locally, enter T. To retrieve from web, enter F: ") == "T" else False

### Load Be star catalog

In [5]:
#Jaschek1982 = v.query_constraints(catalog="III/67A/catalog")[0] # old Be catalog; Table object
BeSS = ascii_io.read("./catalogs/BeSS_catalog.csv", format="csv", header_start=0, data_start=1)
BeSS["_RAJ2000"] = [(float(hms.split()[0])*(360/24)+float(hms.split()[1])/60+float(hms.split()[2])/3600)*u.degree for hms in BeSS["RA"]]
BeSS["_DEJ2000"] = [(float(dms.split()[0])+float(dms.split()[1])/60+float(dms.split()[2])/3600)*u.degree for dms in BeSS["DEC"]]

### Crossmatch with Gaia data

In [6]:
Gaia_BeSS_5as = v.query_region(BeSS, catalog="I/345/gaia2", radius="5s")
Gaia_BeSS_5as[0]

_q,_RAJ2000,_DEJ2000,_r,DR2Name,RA_ICRS,e_RA_ICRS,DE_ICRS,e_DE_ICRS,SolID,Source,RandomI,Epoch,Plx,e_Plx,RPlx,pmRA,e_pmRA,pmDE,e_pmDE,RADEcor,RAPlxcor,RApmRAcor,RApmDEcor,DEPlxcor,DEpmRAcor,DEpmDEcor,PlxpmRAcor,PlxpmDEcor,pmRApmDEcor,NAL,NAC,NgAL,NbAL,gofAL,chi2AL,epsi,sepsi,Solved,APF,WAL,pscol,e_pscol,fvarpi,MatchObsA,Nper,amax,type,MatchObs,Dup,o_Gmag,FG,e_FG,RFG,Gmag,e_Gmag,o_BPmag,FBP,e_FBP,RFBP,BPmag,e_BPmag,o_RPmag,FRP,e_FRP,RFRP,RPmag,e_RPmag,E_BR_RP_,Mode,BP-RP,BP-G,G-RP,RV,e_RV,o_RV,Tefftemp,loggtemp,__Fe_H_temp,Var,GLON,GLAT,ELON,ELAT,fPriam,Teff,b_Teff,B_Teff,AG,b_AG,B_AG,E_BP-RP_,b_E_BP-RP_,B_E_BP-RP_,fFLAME,Rad,b_Rad,B_Rad,Lum,b_Lum,B_Lum,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000
,deg,deg,arcsec,,deg,mas,deg,mas,,,,yr,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / mas2,1 / um,1 / um,,,,mas,,,,,e-/s,e-/s,,mag,mag,,e-/s,e-/s,,mag,mag,,e-/s,e-/s,,mag,mag,,,mag,mag,mag,km / s,km / s,,K,[cm/s2],,,deg,deg,deg,deg,,K,K,K,mag,mag,mag,mag,mag,mag,,Rsun,Rsun,Rsun,Lsun,Lsun,Lsun,deg,mas,deg,mas
int32,float64,float64,float64,bytes28,float64,float64,float64,float64,int64,int64,int64,float32,float64,float32,float64,float64,float32,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,int16,int16,int16,float64,float64,float32,float32,int16,uint8,float64,float64,float32,float32,int16,int16,float64,uint8,int16,uint8,int16,float32,float32,float64,float64,float64,int16,float32,float32,float64,float64,float64,int16,float32,float32,float64,float64,float64,float32,uint8,float64,float64,float64,float64,float32,int16,float32,float32,float32,bytes13,float64,float64,float64,float64,int32,float64,float64,float64,float32,float32,float32,float32,float32,float32,int32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64
1,0.0218798979667,63.5049148930351,3.314,Gaia DR2 431595525500259072,0.02185846183,0.1818,63.50491076662,0.1641,1635721458409799680,431595525500259072,766754377,2015.5,0.2253,0.2222,1.0138,-2.221,0.400,-0.958,0.302,0.0012,0.1528,-0.2532,0.0858,0.1528,0.0682,0.0520,-0.0938,0.2296,0.0211,284,0,280,4,4.5126,394.14,0.797,2.2,31,0,0.2549,1.4441,0.0477,0.046,33,19,0.35687,0,37,0,305,520,1.235,421.054,18.8984,0.0026,27,198.1,7.086,27.960,19.6091,0.0388,28,487.7,6.453,75.580,18.0415,0.0144,1.319,0,1.5676,0.7107,0.8569,--,--,0,--,--,--,NOT_AVAILABLE,117.24198760017,1.20187395588,38.60172532387,55.19040505455,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0.02187989797,6.1984,63.50491489300,4.6770
2,0.0298878479343,60.4500481190305,4.402,Gaia DR2 429343279011625728,0.02986334872,0.4462,60.45004792073,0.4936,1635721458409799680,429343279011625728,1616787697,2015.5,1.0673,0.6179,1.7272,-2.806,0.921,-0.046,0.884,-0.0487,0.0208,-0.2950,-0.0598,0.1980,-0.1273,-0.1509,-0.1200,0.3742,-0.1633,211,0,211,0,-0.4403,196.54,0.000,0,31,0,0.0493,1.5857,0.1417,0.052,25,16,0.87269,0,29,0,238,147.5,0.9087,162.296,20.2665,0.0067,20,56.45,5.628,10.030,20.9722,0.1082,24,167.6,8.99,18.643,19.2013,0.0582,1.519,0,1.7710,0.7057,1.0653,--,--,0,--,--,--,NOT_AVAILABLE,116.64162086075,-1.79343113610,35.07342258739,52.94327619755,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0.02988784793,14.2763,60.45004811900,13.7058
5,0.1065258194996,64.1959216181527,1.598,Gaia DR2 431735545734808704,0.10652581950,2.2098,64.19592161815,1.3005,1635721458409799680,431735545734808704,1437846327,2015.5,--,--,--,--,--,--,--,-0.1801,--,--,--,--,--,--,--,--,--,75,0,74,1,-0.9416,57.92,0.000,0,3,0,0.0160,--,--,0.152,9,7,7.8388,0,11,0,92,73.72,1.256,58.700,21.0195,0.0185,0,--,--,--,--,--,0,--,--,--,--,--,--,0,--,--,--,--,--,0,--,--,--,NOT_AVAILABLE,117.41488307682,1.87196135625,39.50214461654,55.65708620043,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0.10652581950,2.2098,64.19592161820,1.3005
9,0.1954516464357,58.2115127344761,3.472,Gaia DR2 422729647848550784,0.19537601004,0.1070,58.21150182516,0.0983,1635721458409799680,422729647848550784,1528958268,2015.5,0.4441,0.1535,2.8933,-9

### Load the XMM, Chandra, and Swift/XRT catalogs
Depending on whether the catalogs have been previously downloaded this will either requery them from Vizier or use the locally stored copies.

In [10]:
if not local:
    cats = [None, None, None]
    xmatch = [None, None, None]
    cat_ids = ["IX/59/xmm4dr9s","IX/57/csc2master","IX/58/2sxpscle"]
    for i in range(len(cats)):
        cats[i] = v.query_constraints(catalog=cat_ids[i])[0]
        print("Retrieved",cat_ids[i])
        xmatch[i] = v.query_region(Gaia_BeSS_5as[0], catalog=cat_ids[i], radius="5s")
        print("Crossmatched BeSS/Gaia with",cat_ids[i])
    [XMM, CSC, XRT] = cats
    [XMM_xmatch, CSC_xmatch, XRT_xmatch] = xmatch
    
if local:
    with open("./catalogs/xmm4dr9s.pkl","rb") as file:
        XMM = pickle.load(file)
    with open("./catalogs/csc2master.pkl","rb") as file:
        CSC = pickle.load(file)
    with open("./catalogs/2sxpscle.pkl","rb") as file:
        XRT = pickle.load(file)

### Look for coordinate matches between Be stars and x-ray sources

In [12]:
BeSS_coords = SkyCoord(np.array(BeSS["_RAJ2000","_DEJ2000"]), frame='icrs') 
XMM_coords = SkyCoord(np.array(XMM["_RAJ2000","_DEJ2000"]), unit=u.degree, frame='icrs')
CSC_coords = SkyCoord(np.array(CSC["_RAJ2000","_DEJ2000"]), unit=u.degree, frame='icrs')
XRT_coords = SkyCoord(np.array(XRT["_RAJ2000","_DEJ2000"]), unit=u.degree, frame='icrs')

In [21]:
XMM_matches = XMM_coords.match_to_catalog_sky(BeSS_coords)
CSC_matches = CSC_coords.match_to_catalog_sky(BeSS_coords)
XRT_matches = XRT_coords.match_to_catalog_sky(BeSS_coords)

In [22]:
XMM_filt = XMM_matches[1] < 5*u.arcsecond
CSC_filt = CSC_matches[1] < 5*u.arcsecond
XRT_filt = XRT_matches[1] < 5*u.arcsecond

In [28]:
print(XMM[XMM_filt])
print(CSC[CSC_filt])
print(XRT[XRT_filt])

_RAJ2000 _DEJ2000 Source _4XMM RA_ICRS ...  c  uIRAP xcatDBdet xcatDB IRAP
  deg      deg                   deg   ...                                
-------- -------- ------ ----- ------- ... --- ----- --------- ------ ----
_RAJ2000 _DEJ2000 _2CXO RAICRS DEICRS   r0   ... fdw Nac ExpAC Nhrc ExpHRC
  deg      deg           deg    deg   arcsec ...           s          s   
-------- -------- ----- ------ ------ ------ ... --- --- ----- ---- ------
_RAJ2000 _DEJ2000 _2SXPS IAUName RAJ2000 ... _2MASS USNOB1 _2CSC _1SXPS ALLWISE
  deg      deg                     deg   ...                                   
-------- -------- ------ ------- ------- ... ------ ------ ----- ------ -------
